In [1]:
import transformers
print(transformers.__version__)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.50.0


In [2]:
!nvidia-smi

Tue Nov 25 17:41:13 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   58C    P0              72W / 300W |  30216MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/workspace/Approach0


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Dataset Name

In [5]:
dataset_name = "thenlpresearcher/in22_conv_eng_mar_hin"

In [6]:
from datasets import load_dataset
raw_datasets = load_dataset(dataset_name)

In [7]:
# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["src"]
gt_mar     = raw_datasets['test']["mar_Deva"]
gt_hin    = raw_datasets['test']["hin_Deva"]

## Model Names

In [8]:
original_model_name = "ai4bharat/indictrans2-en-indic-dist-200M"
without_punct_model_name = "thenlpresearcher/iitb-en-indic-without-punct"
with_punct_model_name = "thenlpresearcher/iitb-en-indic-only-punct"
combined_x_punct_model_name = "thenlpresearcher/shalaka_fd_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva"
combined_2x_punct_model_name = "thenlpresearcher/shalaka_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva"

### Original

In [9]:
mode = "original"

In [10]:
model_name = original_model_name

In [11]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

import torch

def batch_translate(
    input_sentences,
    src_lang,
    tgt_lang,
    model,
    tokenizer,
    ip,
    device="cuda",
    batch_size=16,
):
    all_translations = []

    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i : i + batch_size]

        # Preprocess (handles entity mapping etc.)
        batch = ip.preprocess_batch(
            batch,
            src_lang=src_lang,
            tgt_lang=tgt_lang
        )

        # Tokenize on device
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        # Generate translations
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode
        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess (restore entities)
        decoded = ip.postprocess_batch(decoded, lang=tgt_lang)

        all_translations.extend(decoded)

    return all_translations

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

2025-11-25 17:41:20.500438: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 17:41:20.562261: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 17:41:22.780769: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
100%|███████████████████████████████████████| 47/47 [03:07<00:00,  4.00s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: आई, उद्या चित्रपटाला जाऊया.
eng_Latn: I don't have to go to school.
mar_Deva: मला शाळेत जाण्याची गरज नाही.
eng_Latn: It is a holiday.
mar_Deva: ती सुट्टी असते.
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: अरे, उद्या 14 एप्रिल आहे ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.
eng_Latn: We can make a movie plan!
mar_Deva: आम्ही चित्रपटाची योजना बनवू शकतो!
eng_Latn: That's a good news!
mar_Deva: ही एक चांगली बातमी आहे!
eng_Latn: Why is it a holiday though?
mar_Deva: मात्र, ही सुट्टी का आहे?
eng_Latn: Are all schools, colleges and offices closed tomorrow?
mar_Deva: सर्व शाळा, महाविद्यालये आणि कार्यालये उद्या बंद आहेत का?
eng_Latn: It is Ambedkar Jayanti tomorrow!
mar_Deva: उद्या आंबेडकर जयंती आहे!


In [12]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

आई, उद्या चित्रपटाला जाऊया.


In [13]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [03:22<00:00,  4.32s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: माँ, चलो कल एक फ़िल्म देखने चलते हैं।
eng_Latn: I don't have to go to school.
mar_Deva: मुझे स्कूल जाने की ज़रूरत नहीं है।
eng_Latn: It is a holiday.
mar_Deva: यह छुट्टी है।
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: ओह, कल 14 अप्रैल है ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: आपके पिता को भी काम से छुट्टी मिलेगी।
eng_Latn: We can make a movie plan!
mar_Deva: हम एक फिल्म की योजना बना सकते हैं!
eng_Latn: That's a good news!
mar_Deva: यह एक अच्छी खबर है!
eng_Latn: Why is it a holiday though?
mar_Deva: लेकिन यह छुट्टी क्यों है?
eng_Latn: Are all schools, colleges and offices closed tomorrow?
mar_Deva: क्या सभी स्कूल, कॉलेज और कार्यालय कल बंद हैं?
eng_Latn: It is Ambedkar Jayanti tomorrow!
mar_Deva: कल अम्बेडकर जयंती है!


In [14]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_hin = remove_prefix(translations, "hin_Deva eng_Latn ")
print(translations_hin[0])

माँ, चलो कल एक फ़िल्म देखने चलते हैं।


In [15]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_conv_eng_mar_hin/original_outputs.csv


In [16]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [17]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 18.94741520242327, CHRF++: 51.04813164303506
Metrics -- Hindi
BLEU: 26.3837176743007 CHRF++: 48.309698872187774


### Without Punctuation Model

In [18]:
mode = "without"

In [19]:
model_name = without_punct_model_name

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [21]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [01:13<00:00,  1.56s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn आई, उद्या चित्रपट बघायला जाऊ.
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मला शाळेत जायची गरज नाही.
eng_Latn: It is a holiday.
mar_Deva: mar_Deva eng_Latn ती सुट्टी असते.
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: mar_Deva eng_Latn अरे, उद्या 14 एप्रिल आहे ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: mar_Deva eng_Latn तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.
eng_Latn: We can make a movie plan!
mar_Deva: mar_Deva eng_Latn आपण चित्रपटाची योजना बनवू शकतो!
eng_Latn: That's a good news!
mar_Deva: mar_Deva eng_Latn ही चांगली बातमी आहे!
eng_Latn: Why is it a holiday though?
mar_Deva: mar_Deva eng_Latn पण ही सुट्टी का आहे?
eng_Latn: Are all schools, colleges and offices closed tomorrow?
mar_Deva: mar_Deva eng_Latn सर्व शाळा, महाविद्यालये आणि कार्यालये उद्या बंद आहेत का?
eng_Latn: It is Ambedkar Jayanti tomorrow!
mar_Deva: mar_Deva eng_Latn उद्या आंबेडकर

In [22]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

आई, उद्या चित्रपट बघायला जाऊ.


In [23]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [05:19<00:00,  6.80s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn माँ, चल कल सिनेमा में जाते हैं..........,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn स्कूल जाने की आवश्यकता ही नहीं है..............................................................................................................................................................................................................................................
eng_Latn: It is a holiday.
mar_Deva: वो एक अवकाश है...........................................................................................................................................................................................................................................................
eng_Latn: Oh, t

In [24]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_conv_eng_mar_hin/without_outputs.csv


In [25]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [26]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 16.674078534665142, CHRF++: 49.68337569456086
Metrics -- Hindi
BLEU: 26.3837176743007 CHRF++: 48.309698872187774


### With Punctuation Model

In [27]:
mode = "with"

In [28]:
model_name = with_punct_model_name

In [29]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [30]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [01:54<00:00,  2.44s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn आई, उद्या चित्रपट बघायला जाऊ.
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मला शाळेत जाण्याची गरज नाही.
eng_Latn: It is a holiday.
mar_Deva: mar_Deva eng_Latn ती सुट्टी असते.
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: mar_Deva eng_Latn अरे, उद्या 14 एप्रिल आहे ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: mar_Deva eng_Latn तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.
eng_Latn: We can make a movie plan!
mar_Deva: mar_Deva eng_Latn आपण चित्रपटाची योजना बनवू शकतो!
eng_Latn: That's a good news!
mar_Deva: mar_Deva eng_Latn ही चांगली बातमी आहे!
eng_Latn: Why is it a holiday though?
mar_Deva: mar_Deva eng_Latn पण ही सुट्टी का आहे?
eng_Latn: Are all schools, colleges and offices closed tomorrow?
mar_Deva: mar_Deva eng_Latn सर्व शाळा, महाविद्यालये आणि कार्यालये उद्या बंद आहेत का?
eng_Latn: It is Ambedkar Jayanti tomorrow!
mar_Deva: mar_Deva eng_Latn उद्या आंबे

In [31]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

आई, उद्या चित्रपट बघायला जाऊ.


In [32]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [04:05<00:00,  5.22s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn माँ, चलो कल सिनेमा के लिए जाते हैं.
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मुझे स्कूल नहीं जाना है.
eng_Latn: It is a holiday.
mar_Deva: mar_Deva eng_Latn वह एक अवकाश है।
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: mar_Deva eng_Latn ओह, कल 14 एप्रिल है ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: mar_Deva eng_Latn आपके पिता को भी काम से एक दिन की अवकाश मिल रहा है.
eng_Latn: We can make a movie plan!
mar_Deva: mar_Deva eng_Latn हम एक फिल्म की योजना बना सकते हैं!
eng_Latn: That's a good news!
mar_Deva: mar_Deva eng_Latn ये एक खुश खबर है!
eng_Latn: Why is it a holiday though?
mar_Deva: mar_Deva eng_Latn फिर भी ये अवकाश क्यों है?
eng_Latn: Are all schools, colleges and offices closed tomorrow?
mar_Deva: mar_Deva eng_Latn सब स्कूल, कॉलेज और ऑफिस कल बंद होते हैं?
eng_Latn: It is Ambedkar Jayanti tomorrow!
mar_Deva: mar_Deva eng_Latn कल आंबेडकर जयंत

In [33]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})


import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_conv_eng_mar_hin/with_outputs.csv


In [34]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [35]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 16.078959081533664, CHRF++: 49.770554463446146
Metrics -- Hindi
BLEU: 26.3837176743007 CHRF++: 48.309698872187774


### Combined Punctuation Model (2x)

In [36]:
mode = "combined_2x"

In [37]:
model_name = combined_2x_punct_model_name

In [38]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [39]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [02:06<00:00,  2.69s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn आई, उद्या एखाद्या चित्रपटाला जाऊ.
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मला शाळेत जाण्याची गरज नाही.
eng_Latn: It is a holiday.
mar_Deva: mar_Deva eng_Latn ती सुट्टी असते.
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: mar_Deva eng_Latn अरे, उद्या 14 एप्रिल आहे ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: mar_Deva eng_Latn तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.
eng_Latn: We can make a movie plan!
mar_Deva: mar_Deva eng_Latn आपण चित्रपटाची योजना बनवू शकतो!
eng_Latn: That's a good news!
mar_Deva: mar_Deva eng_Latn ही चांगली बातमी आहे!
eng_Latn: Why is it a holiday though?
mar_Deva: mar_Deva eng_Latn पण ती सुट्टी का असते?
eng_Latn: Are all schools, colleges and offices closed tomorrow?
mar_Deva: mar_Deva eng_Latn सर्व शाळा, महाविद्यालये आणि कार्यालये उद्या बंद आहेत का?
eng_Latn: It is Ambedkar Jayanti tomorrow!
mar_Deva: mar_Deva eng_Latn उद्या

In [40]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

आई, उद्या एखाद्या चित्रपटाला जाऊ.


In [41]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [06:00<00:00,  7.68s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn माँ, कल फिल्म देखने चलते हैं हम.............................................................................................................................................................................................................................................
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मुझे स्कूल जाने की कोई आवश्यकता नहीं है.............................................................................................................................................................................................................................................
eng_Latn: It is a holiday.
mar_Deva: यह एक अवकाश है।
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: mar_Deva eng_Latn ओह, कल 14अप्रिल है, है ना?..........................................................................................................................................................

In [42]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_conv_eng_mar_hin/combined_2x_outputs.csv


In [43]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [44]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 17.930452115794093, CHRF++: 50.44942453145798
Metrics -- Hindi
BLEU: 26.3837176743007 CHRF++: 48.309698872187774


### Combined Punctuation Model (x)

In [45]:
mode = "combined_x"

In [46]:
model_name = combined_x_punct_model_name

In [47]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [48]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [02:57<00:00,  3.78s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn आई, उद्या एखाद्या चित्रपटाला जाऊ.
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मला शाळेत जाण्याची गरज नाही.
eng_Latn: It is a holiday.
mar_Deva: mar_Deva eng_Latn ती सुट्टी असते.
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: mar_Deva eng_Latn अरे, उद्या 14 एप्रिल आहे ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: mar_Deva eng_Latn तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.
eng_Latn: We can make a movie plan!
mar_Deva: mar_Deva eng_Latn आपण चित्रपटाची योजना बनवू शकतो!
eng_Latn: That's a good news!
mar_Deva: mar_Deva eng_Latn ही चांगली बातमी आहे!
eng_Latn: Why is it a holiday though?
mar_Deva: मात्र, ती सुट्टी का असते?
eng_Latn: Are all schools, colleges and offices closed tomorrow?
mar_Deva: mar_Deva eng_Latn सर्व शाळा, महाविद्यालये आणि कार्यालये उद्या बंद आहेत का?
eng_Latn: It is Ambedkar Jayanti tomorrow!
mar_Deva: mar_Deva eng_Latn उद्या आंबेडकर जयंती

In [49]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

आई, उद्या एखाद्या चित्रपटाला जाऊ.


In [50]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [06:22<00:00,  8.13s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn माँ, चलो कल एक फिल्म देखने जाते हैं। "
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मुझे स्कूल जाने की कोई आवश्यकता नहीं है।
eng_Latn: It is a holiday.
mar_Deva: यह छुट्टी का दिन होता है।
eng_Latn: Oh, tomorrow is the 14th of April right?
mar_Deva: mar_Deva eng_Latn ओह, कल 14 अप्रैल की तारीख है, है ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: mar_Deva eng_Latn आपके पिता को भी काम से एक दिन की छुट्टी मिलनी तय है।
eng_Latn: We can make a movie plan!
mar_Deva: हम एक फिल्म की योजना बना सकते हैं!
eng_Latn: That's a good news!
mar_Deva: ये तो एक अच्छी खबर है.........................................................................................................................................................................................................................................................
eng_Latn: Why is it a holiday though?
mar_Deva: फिर भी, यह छुट्टी क्य

In [51]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_conv_eng_mar_hin/combined_x_outputs.csv


In [52]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [53]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 17.874171775737672, CHRF++: 50.339241451649706
Metrics -- Hindi
BLEU: 26.3837176743007 CHRF++: 48.309698872187774


## T5

In [54]:
model_name = original_model_name
mode = "t5"

In [55]:
from transformers import pipeline

# Use batching + GPU (if available)
punctuator_pipeline = pipeline(
    "text2text-generation",
    model="thenlpresearcher/iitb-t5-finetuned-punctuation",
    device=device,            # GPU; use device=-1 for CPU
    batch_size=64        # adjust based on GPU RAM
)

def restore_punctuation_t5_batch(text_list):
    # The pipeline automatically batches under the hood
    outputs = punctuator_pipeline(
        text_list,
        max_length=128
    )
    # Pipeline returns list of dicts
    return [o["generated_text"] for o in outputs]

Device set to use cuda


In [56]:
# Run the whole batch in parallel
predicted_sentences = restore_punctuation_t5_batch(src_sentences)
print(len(predicted_sentences))

1503


In [57]:
input_sentences = predicted_sentences

In [58]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [03:05<00:00,  3.95s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn आई, उद्या एखाद्या चित्रपटाला जाऊ.
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मला शाळेत जाण्याची गरज नाही.
eng_Latn: It is a holiday.
mar_Deva: mar_Deva eng_Latn ती सुट्टी असते.
eng_Latn: Oh, tomorrow is the 14th of April, right?
mar_Deva: mar_Deva eng_Latn अरे, उद्या 14 एप्रिल आहे ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: mar_Deva eng_Latn तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.
eng_Latn: We can make a movie plan!
mar_Deva: mar_Deva eng_Latn आपण चित्रपटाची योजना बनवू शकतो!
eng_Latn: That's a good news!
mar_Deva: mar_Deva eng_Latn ही चांगली बातमी आहे!
eng_Latn: Why is it a holiday though?
mar_Deva: मात्र, ती सुट्टी का असते?
eng_Latn: Are all schools, colleges and offices closed tomorrow?
mar_Deva: mar_Deva eng_Latn सर्व शाळा, महाविद्यालये आणि कार्यालये उद्या बंद आहेत का?
eng_Latn: It is Ambedkar Jayanti tomorrow!
mar_Deva: mar_Deva eng_Latn उद्या आंबेडकर जयंत

In [59]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

आई, उद्या एखाद्या चित्रपटाला जाऊ.


In [60]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 47/47 [06:17<00:00,  8.03s/it]

eng_Latn: Mom, let's go for a movie tomorrow.
mar_Deva: mar_Deva eng_Latn माँ, चलो कल एक फिल्म देखने जाते हैं। "
eng_Latn: I don't have to go to school.
mar_Deva: mar_Deva eng_Latn मुझे स्कूल जाने की कोई आवश्यकता नहीं है।
eng_Latn: It is a holiday.
mar_Deva: यह छुट्टी का दिन होता है।
eng_Latn: Oh, tomorrow is the 14th of April, right?
mar_Deva: mar_Deva eng_Latn ओह, कल 14 अप्रैल की तारीख है, है ना?
eng_Latn: Your dad will also have the day off from work.
mar_Deva: mar_Deva eng_Latn आपके पिता को भी काम से एक दिन की छुट्टी मिलनी तय है।
eng_Latn: We can make a movie plan!
mar_Deva: हम एक फिल्म की योजना बना सकते हैं!
eng_Latn: That's a good news!
mar_Deva: ये तो एक अच्छी खबर है.........................................................................................................................................................................................................................................................
eng_Latn: Why is it a holiday though?
mar_Deva: फिर भी, यह छुट्टी क्

In [61]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_conv_eng_mar_hin/t5_outputs.csv


In [62]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [63]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 17.822822344140665, CHRF++: 50.23633412097026
Metrics -- Hindi
BLEU: 26.3837176743007 CHRF++: 48.309698872187774
